In [ ]:
%pylab inline
import numpy as np
import scipy.stats as spstats
import pandas as pd

In [ ]:
out_dirs = [
    "archive/paper-2.0/nk_rewire/2017-09-21 09:38:29 5dbaa49 1",
    "archive/paper-2.0/nk_rewire/2017-09-21 09:46:02 5dbaa49 2",
    "archive/paper-2.0/nk_rewire/2017-09-21 10:03:30 5dbaa49 3",
    "archive/paper-2.0/nk_rewire/2017-09-21 10:30:07 5dbaa49 4"
]

In [ ]:
df_runs = pd.concat([pd.DataFrame.from_csv("%s/runs.csv" % d) for d in out_dirs])
df_values = pd.concat([pd.DataFrame.from_csv("%s/values.csv" % d) for d in out_dirs])
rs = sorted(set((df_values["rewire"] * 6).round() / 6.0))
strategies = ["best", "loc_best", "conform", "loc_conform", "loc_cons"]
trials = sorted(set(df_values["trial"]))
steps = sorted(set(df_values["step"]))

In [ ]:
plt.figure(figsize=(1*15.0/6.0,1*15.0/6.0))
for i, r in enumerate(rs):
    df_rewire = df_values[df_values["rewire"] == r]
    for j, strategy in enumerate(["loc_best"]): #enumerate(strategies):
        df_strategy = df_rewire[df_rewire["strategy"] == strategy]
        #plt.subplot(len(rs),len(strategies),1+j+i*len(strategies))
        plt.title(strategy)
        plt.ylabel("Rewire = %0.2f" % r)
        for trial_n in sorted(set(df_strategy["trial"]))[:3]:
            df_trial = df_strategy[df_strategy["trial"] == trial_n]
            plot(df_trial["step"], df_trial["value"])
            xlabel("Iteration")
            ylabel("Value")
plt.tight_layout()
plt.savefig("fig-trials.png", format="png", dpi=300)

In [ ]:
def values_to_efficiency(values):
    max_delta = [max(values) - values[i] for i in range(len(values))]
    nonpositive = [i for i, d in enumerate(max_delta) if d <= 0.0001]
    return 1.0 / float(nonpositive[0])

def values_to_converge(values):
    max_delta = [max(values) - values[i] for i in range(len(values))]
    nonpositive = [i for i, d in enumerate(max_delta) if d <= 0.0001]
    return nonpositive[0]    

In [ ]:
d_trial = []
d_strat = []
d_eff = []
d_perf = []
d_rewire = []
d_degree = []
d_converge = []
for i, trial in enumerate(sorted(set(df_values["trial"]))):
    for j, strategy in enumerate(strategies):
        df_strat = df_values[df_values["strategy"] == strategy]
        df = df_strat[df_strat["trial"] == trial]
        eff = values_to_efficiency(list(df["value"]))
        d_converge.append(values_to_converge(list(df["value"])))
        d_trial.append(trial)
        d_strat.append(strategy)
        d_eff.append(eff)
        d_rewire.append(df_runs["rewire"][trial])
        d_degree.append(df_runs["degree"][trial])
        d_perf.append(df_runs["%s_perf" % strategy][trial])
df_eff = pd.DataFrame({
    "trial": d_trial,
    "rewire": d_rewire,
    "degree": d_degree,
    "strategy": d_strat,
    "eff": d_eff,
    "perf": d_perf,
    "converge": d_converge
})

In [ ]:
plt.figure(figsize=(3*15.0/6.0,2*15.0/6.0))
for i, strategy in enumerate(strategies):
    plt.subplot(2,3, i+1)
    plt.title(strategy)
    df = df_eff[df_eff["strategy"] == strategy]
    plot(df["converge"], df["perf"], '.')
    xlabel('Converge Time')
    ylabel('Value')
    xlim([0, 300])
    ylim([0.7, 0.8])
plt.tight_layout()
plt.savefig("fig-converge-value.png", format="png", dpi=300)

In [ ]:
plt.figure(figsize=(4*4,4*2))
for j, strategy in enumerate(strategies):
    df_strategy = df_values[df_values["strategy"] == strategy]
    plt.subplot(2,4,1+j)
    plt.title("Strategy: %s" % strategy)
    for i, r in enumerate(rs):
        df_rewire = df_strategy[(df_strategy["rewire"] - r).abs() < 0.01]
        label = "Rewire = %0.2f" % r
        step_mean = []
        step_se = []
        for step in sorted(set(df_rewire["step"])):
            df = df_rewire[df_rewire["step"] == step]
            step_mean.append(df["value"].mean())
            s = df["value"].std(ddof=1)
            step_se.append(s / sqrt(len(df["value"])))
        mean = np.array(step_mean)
        se = np.array(step_se)
        plt.fill_between(range(len(step_mean)), mean - se, mean + se, facecolor="#dfdfff")
        plt.plot(step_mean, label=label)
    plt.legend()
    plt.grid()
    plt.ylim([0.5, 0.8])
    plt.ylim([0.72, 0.76])
plt.tight_layout()

In [ ]:
df = df_rewire[df_rewire["strategy"] == "loc_conform"]

In [ ]:
df.head()

In [ ]:
fs = 12
tfs = 10
strat_data = {
    "best": {
        "color": "#aaaa00",
        "fmt": ":",
        "label": "Best+I"
    },
    "conform": {
        "color": "#aaaa00",
        "fmt": "--",
        "label": "Conf+I"
    },
    "loc_best": {
        "color": "#000066",
        "fmt": ":",
        "label": "Best+LI"
    },
    "loc_conform": {
        "color": "#000066",
        "fmt": "--",
        "label": "Conf+LI"
    },
    "loc_cons": {
        "color": "#000000",
        "fmt": "-",
        "label": "LMaj+I"
    },
}
plt.figure(figsize=(15/3.0,7.5/3.0))
for i, r in enumerate([0, 1]):
    df_rewire = df_values[(df_values["rewire"] - r).abs() < 0.01]
    plt.subplot(1,2,1+i)
    plt.title("P[rewire] = %0.2f" % r, fontsize=fs)
    for j, strategy in enumerate(strategies):
        df_strategy = df_rewire[df_rewire["strategy"] == strategy]
        label = strategy
        step_mean = []
        step_se = []
        for step in sorted(set(df_strategy["step"])):
            df = df_strategy[df_strategy["step"] == step]
            step_mean.append(df["value"].mean())
            s = df["value"].std(ddof=1)
            step_se.append(s / sqrt(len(df["value"])))
        mean = np.array(step_mean)
        se = np.array(step_se)
        #plt.fill_between(range(len(step_mean)), mean - se, mean + se, facecolor="#dfdfff")
    for j, strategy in enumerate(strategies):
        df_strategy = df_rewire[df_rewire["strategy"] == strategy]
        label = strategy
        step_mean = []
        step_se = []
        for step in sorted(set(df_strategy["step"])):
            df = df_strategy[df_strategy["step"] == step]
            step_mean.append(df["value"].mean())
            s = df["value"].std(ddof=1)
            step_se.append(s / sqrt(len(df["value"])))
        mean = np.array(step_mean)
        se = np.array(step_se)
        plt.plot(
            step_mean,
            linestyle=strat_data[strategy]["fmt"],
            color=strat_data[strategy]["color"],
            label=strat_data[strategy]["label"],
            linewidth=1)
    xlabel("Iteration", fontsize=fs)
    ylabel("Value", fontsize=fs)
    plt.grid()
    plt.legend(fontsize=tfs)
    plt.ylim([0.5, 0.75])
    plt.tick_params(axis='both', which='major', labelsize=tfs)
plt.tight_layout()
plt.savefig("fig-val-iter.pdf", format="pdf")
plt.savefig("fig-val-iter.png", format="png", dpi=300)

In [ ]:
strategies = sorted(set(df_eff["strategy"]))
rw = sorted(set(df_eff["rewire"]))

for s in strategies:
    for r in [0]:
        d = df_eff[(df_eff["strategy"] == s) & (df_eff["rewire"] == r)]
        print(s)
        print("  ", d["eff"].mean(), d["eff"].std(ddof=1) / math.sqrt(len(d)))
        print("  ", d["perf"].mean(), d["perf"].std(ddof=1) / math.sqrt(len(d)))

In [ ]:
def estimate_curve(data_x, data_y, points, window):
    h = max(data_x)
    l = min(data_x)
    step = (h-l)/float(points - 1)
    curve_x = np.arange(points) * step + l
    curve_y = np.ones(points)
    for j, x in enumerate(curve_x):
        total = 0.0
        norm = 0.0
        for i, dx in enumerate(data_x):
            dy = data_y[i]
            total += dy * np.exp(-1 * np.power(x - dx, 2) / (2.0*window))
            norm += np.exp(-1 * np.power(x - dx, 2) / (2.0*window))
        curve_y[j] = total / norm
    return curve_x, curve_y

def line_points(m, b, x):
    x0 = min(x)
    x1 = max(x)
    y0 = m*x0 + b
    y1 = m*x1 + b
    return [x0,x1], [y0,y1]

In [ ]:
fs = 14
tfs = 10
param = "degree"
plt.figure(figsize=(75/6.0,15/6.0))
for j, strategy in enumerate(strategies):
    plt.subplot(1,5,1+j)
    r,p = spstats.pearsonr(df_runs[param], df_runs["%s_perf" % strategy])
    plt.plot(df_runs[param], df_runs["%s_perf" % strategy], '.', markersize=2)
    m, b = np.polyfit(df_runs[param], df_runs["%s_perf" % strategy], 1)
    #x, y = estimate_curve(df_runs["degree"], df_runs["%s_perf" % strategy], 100, 5)
    x, y = line_points(m,b,df_runs[param])
    plt.plot(x, y, '-')
    plt.title(strat_data[strategy]["label"], fontsize=fs)
    print "%s r=%0.2f p=%0.4f m=%f, b=%f" % (strategy, r, p, m, b)
    xlabel("Degree", fontsize=fs)
    ylabel("Performance", fontsize=fs)
    plt.grid()
    if strategy == "loc_conform":
        plt.ylim([0.55, 0.62])
    else:
        plt.ylim([0.68, 0.77])
    plt.tick_params(axis='both', which='major', labelsize=tfs)
plt.tight_layout()
plt.savefig("fig-deg-perf.pdf", format="pdf")
plt.savefig("fig-deg-perf.png", format="png", dpi=300)

In [ ]:
plt.figure(figsize=(5.0*15.0/6.0,15.0/6.0))
for j, strategy in enumerate(strategies):
    plt.subplot(1,5,1+j)
    r,p = spstats.pearsonr(df_runs["rewire"], df_runs["%s_perf" % strategy])
    mean = []
    serr = []
    std = []
    for rw in rs:
        df = df_runs[(df_runs["rewire"] - rw).abs() < 0.01]
        m = df["%s_perf" % strategy].mean()
        s = df["%s_perf" % strategy].std(ddof=1)
        se = s / sqrt(float(len(df["%s_perf" % strategy])))
        mean.append(m)
        serr.append(se)
        std.append(s)
    plt.errorbar(rs, mean, yerr=se)
    plt.title(strategy)
    print "%s r=%0.2f p=%0.4f" % (strategy, r, p)
    xlabel("P[rewire]")
    ylabel("Performance")
    plt.legend()
    plt.grid()
    plt.ylim([0.72, 0.76])
plt.tight_layout()

In [ ]:
fs = 14
tfs = 10
param = "degree"
plt.figure(figsize=(75/6.0,15/6.0))
for j, strategy in enumerate(strategies):
    plt.subplot(1,5,1+j)
    df = df_eff[df_eff["strategy"] == strategy]
    r,p = spstats.pearsonr(df[param], df["eff"])
    m, b = np.polyfit(df[param], df["eff"], 1)
    #x, y = estimate_curve(df_strat["degree"], df_strat["eff"], 100, 5)
    x, y = line_points(m,b,df[param])
    plt.plot(df[param], df["eff"], '.', markersize=2)
    plt.plot(x, y)
    plt.title(strat_data[strategy]["label"], fontsize=fs)
    print "%s r=%0.2f p=%0.4f m=%f, b=%f" % (strategy, r, p, m, b)
    xlabel("Degree", fontsize=fs)
    ylabel("Efficiency", fontsize=fs)
    plt.grid()
    if strategy == "loc_conform":
        pass#plt.ylim([0.55, 0.62])
    else:
        pass#plt.ylim([0.68, 0.77])
    plt.tick_params(axis='both', which='major', labelsize=tfs)
plt.tight_layout()
plt.savefig("fig-deg-eff.pdf", format="pdf")
plt.savefig("fig-deg-eff.png", format="png", dpi=300)


In [ ]:
plt.figure(figsize=(5.0*15.0/6.0,15.0/6.0))
for j, strategy in enumerate(strategies):
    plt.subplot(1,5,1+j)
    df_strat = df_eff[df_eff["strategy"] == strategy]
    r,p = spstats.pearsonr(df_strat["rewire"], df_strat["eff"])
    mean = []
    serr = []
    std = []
    for rw in rs:
        df = df_strat[(df_strat["rewire"] - rw).abs() < 0.01]
        m = df["eff"].mean()
        s = df["eff"].std(ddof=1)
        se = s / sqrt(float(len(df["eff"])))
        mean.append(m)
        serr.append(se)
        std.append(s)
    m, b = np.polyfit(df_strat["degree"], df_strat["eff"], 1)
    #x, y = estimate_curve(df_strat["degree"], df_strat["eff"], 100, 5)
    x, y = line_points(m,b,df_strat["degree"])
    plt.errorbar(sorted(set(df_strat["rewire"])), mean, yerr=se)
    plt.title("%s r=%0.2f p=%0.4f" % (strategy, r, p))
    plt.legend()
    plt.grid()
    plt.ylim([0.005, 0.031])
plt.tight_layout()


In [ ]:
plt.plot(df_runs["degree"], df_runs["meanpath"], '.')

In [ ]:
m = df_runs["degree"].mean()
s = df_runs["degree"].std()
m, s

In [ ]:
m = df_runs["meanpath"].mean()
s = df_runs["meanpath"].std()
m, s

In [ ]:
plt.hist(df_runs["degree"])